<a href="https://colab.research.google.com/github/jegilj/GMM-from-scratch/blob/master/Ajuste_de_hiperpar%C3%A1metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ajuste de hiperparámetros**

*Máster en Ciencia de Datos y Aprendizaje Automático - Universidad de La Rioja*

En este notebook practicaremos los distintos métodos de búsqueda automática de los mejores hiperparámetros vistos en la sesión de teoría.



# Obtención de los datos

Vamos a utilizar la base de datos Diabetes del repositorio StatLib. Es una base de datos habitual para hacer pruebas. El objetivo es construir un modelo que sea capaz de predecir la progresión de la diabetes a un año a partir de carácterísticas personales y datos de analíticas de sangre.

Cargamos directamente la BD desde los datasets de sklearn.
La base de datos contiene información sobre 442 pacientes de 10 variables diferentes. Podemos encontrar más información sobre la base de datos en [[1]](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) [[2]](https://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf).

In [1]:
from sklearn import datasets

diabetes_BD = datasets.load_diabetes()
print(diabetes_BD.DESCR)


.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

Esta base de datos tiene variables como la edad, el sexo, el índice de masa corporal, la presión arterial y otras derivadas de la analítica en sangre. Queremos inferir una variable cuantitativa que representa la progresión de la enfermedad a un año. Se trata por lo tanto de un problema de **regresión**.

Miramos las primeras filas de la base de datos.

In [2]:
import pandas as pd
df=pd.DataFrame(data=diabetes_BD.data,columns=diabetes_BD.feature_names)
target=pd.DataFrame(data=diabetes_BD.target,columns=['target'])
df=pd.concat([df,target],sort=True,axis=1)
df.head(15)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0
5,-0.092695,-0.044642,-0.040696,-0.019442,-0.068991,-0.079288,0.041277,-0.076395,-0.041176,-0.096346,97.0
6,-0.045472,0.050680,-0.047163,-0.015999,-0.040096,-0.024800,0.000779,-0.039493,-0.062917,-0.038357,138.0
7,0.063504,0.050680,-0.001895,0.066629,0.090620,0.108914,0.022869,0.017703,-0.035816,0.003064,63.0
8,0.041708,0.050680,0.061696,-0.040099,-0.013953,0.006202,-0.028674,-0.002592,-0.014960,0.011349,110.0
9,-0.070900,-0.044642,0.039062,-0.033213,-0.012577,-0.034508,-0.024993,-0.002592,0.067737,-0.013504,310.0


Las columnas son numéricas y no contienen valores nulos. Además, como indican en la información de la BD, las variables están estandarizadas (media 0 y varianza 1).

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


Separamos las variables que usaremos para inferir de la variable objetivo.

In [4]:
diabetes=diabetes_BD["data"] # Datos de a usar
feature_names=diabetes_BD["feature_names"] # Nombre de las variables
diabetes_target=diabetes_BD["target"] # Dato a inferir.

In [5]:
diabetes.shape

(442, 10)

In [6]:
from scipy import stats
stats.describe(diabetes_target)

DescribeResult(nobs=442, minmax=(25.0, 346.0), mean=152.13348416289594, variance=5943.331347923785, skewness=0.43906639932477265, kurtosis=-0.8866436055681386)

Fijarse que la progresión media de la enfermedad es de unos 152 puntos con un mínimo de unos 25 y un máximo de 346.

# División en conjunto de entrenamiento y conjunto de test

Como se ha comentado en otras sesiones, el preprocesamiento puede tener un efecto muy grande en el rendimiento del modelo. Esta base de datos ya está preprocesada (variables estandarizadas, sin nulos...).

Dividimos el conjunto entrenamiento y test (20%)

In [7]:
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split(diabetes, diabetes_target, test_size=0.2, random_state=42)

In [8]:
X_test.shape

(89, 10)

In [9]:
X_train.shape

(353, 10)

# Definición y entrenamiento de un modelo con los hiperparámetros por defecto

Elelgimos tres modelos: SVM, árbol de decisión y un random forest.

Definimos primero los modelos de SVM para regresión ([SVR](https://towardsdatascience.com/an-introduction-to-support-vector-regression-svr-a3ebc1672c2)) y árbol de decisión para regresión ([decision tree regressor](https://dev.to/nexttech/classification-and-regression-analysis-with-decision-trees-jgp)).

In [10]:
from sklearn.svm import SVR
svm_reg = SVR()
svm_reg.get_params() #Obtiene los valores de los hiperparámetros que se han usado al definir el modelo

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [11]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.get_params() #Obtiene los valores de los hiperparámetros que se han usado al definir el modelo

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

## Validación cruzada de los modelos

Primero el SVM.

Usamos validación cruzada con k=3. (Nota: Habitualmente se usa una k=10. Usamos k=3 por motivo de que los modelos se entrenen más rápido en el laboratorio).

El conjunto de entrenamiento se divide en 3 partes de forma aleatoria, se entrena con 2 de ellas y se hace test con la tercera. El resultado es un array con 3 resultados.

In [12]:
import numpy as np

from sklearn.model_selection import cross_val_score

svm_scores = cross_val_score(svm_reg, X_train, y_train,
                             scoring="neg_mean_squared_error", cv=3)
svm_rmse_scores = np.sqrt(-svm_scores)

La función de cross-validation de Scikit-Learn espera una función que marque que mayor es mejor en lugar de una funión de coste (menor es mejor), por ello la función score es el opuesto de del Error Cuadrático Medio (MSE). Así, el código anterior calcula -scores antes de hacer la raíz cuadrada del error cuadrático medio. Recordar que el RMSE nos permite interpretar el error cometido en las mismas unidades de la variable objetivo.

Como tenemos 3 folds podemos hacer la media y calcular la desviación típica de los resultados.

In [13]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
display_scores(svm_rmse_scores)

Scores: [75.44007052 74.32643734 71.84312515]
Mean: 73.86987766878676
Standard deviation: 1.5035156690198692


Obtenermos un error medio cercano a 74 puntos.

Ahora, del árbol de decisión

In [14]:
scores = cross_val_score(tree_reg, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=3)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

Scores: [76.29903258 82.94489696 82.48973   ]
Mean: 80.57788651502045
Standard deviation: 3.031307471359646


Obtenemos un resultado algo peor.

Tratamos de crear un Random Forest. Los fundamentos los explicaremos en el tema siguiente. Por ahora, saber simplemente que es un conjunto de árboles de decisión.



In [15]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42)
forest_reg.get_params() #Obtiene los valores de los hiperparámetros que se han usado al definir el modelo

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [16]:
scores = cross_val_score(forest_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=3)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [56.11013477 64.07362286 58.65154318]
Mean: 59.61176693581162
Standard deviation: 3.3212254668233063


Mejora el resultado con un error medio de aproximadamente 60 puntos.

# Ajustar los parámetros del modelo

# Grid Search

En lugar de probar parámetros manualmente, podemos indicar con qué hiperparámetros queremos experimentar y qué valores para ellos queremos probar. El método de búsqueda por cuadrícula evaluará todas las posibles combinaciones, usando validación cruzada.


En el siguiente código se busca la mejor combinación para los parámetros 'n_estimators' con [30, 100, 300] y  'max_features' con [3, 6, 8, 'auto'] para los hiperparámetros de un RandomForestRegressor.
Por defecto, ha usado antes 'n_estimators=100' y 'max_features=auto'

La búsqueda explorará 12 combinaciones del RandomForestRegressor, entrenará el modelo 3 veces (ya que hemos indicado 3-fold cross validation). En otras palabras, tendremos 12 × 3 = 36
rondas de entrenamiento. Puede llevarle un tiempo, pero obtendremos la mejor combinación de hiperparámetros.

Habitualmente se elige validación cruzada con 10 folds (o con 5). Elegimos aquí 3 para acelerar un poco los algoritmos en este laboratorio.

In [17]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid = {
    # 12 (3×4) combinaciones de hiperparámetros
    'n_estimators': [30, 100, 300],
    'max_features': [3, 6, 8, 'auto']
    }

forest_reg = RandomForestRegressor(random_state=42)
# entrena con 3 folds, es decir un total de 12*3=36 rondas de entrenamiento
grid_search_RF = GridSearchCV(forest_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           refit= True, verbose=2, n_jobs=-1)
grid_search_RF.fit(X_train, y_train) # Reentrena el mejor método obtenido

Fitting 3 folds for each of 12 candidates, totalling 36 fits
CPU times: user 1.26 s, sys: 102 ms, total: 1.36 s
Wall time: 30.8 s


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_features': [3, 6, 8, 'auto'],
                         'n_estimators': [30, 100, 300]},
             scoring='neg_mean_squared_error', verbose=2)

Podemos acceder a la mejor combinación de hiperparámetros en *best_params_*:

In [18]:
grid_search_RF.best_params_

{'max_features': 3, 'n_estimators': 300}

También se pode acceder al mejor modelo directamente en *best_estimator_*:

In [19]:
grid_search_RF.best_estimator_

RandomForestRegressor(max_features=3, n_estimators=300, random_state=42)

Como el parámetro **refit** de GridSearchCV tiene el valor True (por defecto), entonces una vez encuentra el mejor modelo usando validación cruzada, se reentrena el mejor modelo con todos los datos de entrenamiento.

Accedemos al resultado del mejor modelo en *best_score_*

In [20]:
negative_mse = grid_search_RF.best_score_
rmse = np.sqrt(-negative_mse)
rmse

57.22156826612568

Podemos acceder al resultado de todos los estimadores testados durante la búsqueda por cuadrícula:

In [21]:
cvres = grid_search_RF.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

58.8767359530435 {'max_features': 3, 'n_estimators': 30}
57.48769464759412 {'max_features': 3, 'n_estimators': 100}
57.22156826612568 {'max_features': 3, 'n_estimators': 300}
59.38212276764398 {'max_features': 6, 'n_estimators': 30}
58.458407717722665 {'max_features': 6, 'n_estimators': 100}
57.98726274576028 {'max_features': 6, 'n_estimators': 300}
60.63270576775128 {'max_features': 8, 'n_estimators': 30}
59.31727479986947 {'max_features': 8, 'n_estimators': 100}
58.657594123944776 {'max_features': 8, 'n_estimators': 300}
60.11032228855047 {'max_features': 'auto', 'n_estimators': 30}
59.7042150589973 {'max_features': 'auto', 'n_estimators': 100}
58.95332896423225 {'max_features': 'auto', 'n_estimators': 300}


En este ejemplo, obtenemos la mejor solución para los hiperparámtros max_features = 3, n_estimators= 300. El RMSE para esta combinación es aproximadamente 57 puntos, que es mejor que la que obteníamos en nuestro primer intento.

## Ejercicios Grid Search.

Trataremos de buscar una combinación adecuada de hiperparámetros para un modelo Support Vector Machine Regressor (`sklearn.svm.SVR`). Definimos a continuación el modelo.  

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
svm_reg = SVR()

PREGUNTA: ¿Qué parámetros ha utilizado por defecto el modelo?

In [32]:
#Solución
svm_reg.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

EJERCICIO:
Utilizar el métod de GridSearch para encontrar la mejor combinación de hiperparámetros para el hiperparámetro 'kernel', 'C' y 'gamma'.

En particular para:

- 'kernel': ['linear'], 'C': [1.0, 3.0];

o

- 'kernel': ['rbf'], 'C': [1.0, 3.0, 5.0],        'gamma': [0.01, 0.1, 1.0]


(Poner entre corchetes [] dos listas {} separadas por una coma)

Usa cross validation con tres folds.

In [33]:
#SOLUCIÓN
param_grid = {
    'kernel': ['linear'],
    'C': [1.0, 3.0]
    }

grid_search_SVR = GridSearchCV(svm_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           refit= True, verbose=2, n_jobs=-1)
grid_search_SVR.fit(X_train, y_train) # Reentrena el mejor método obtenido

Fitting 3 folds for each of 2 candidates, totalling 6 fits


GridSearchCV(cv=3, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1.0, 3.0], 'kernel': ['linear']},
             scoring='neg_mean_squared_error', verbose=2)

In [39]:
#SOLUCIÓN
param_grid_2 = {
    'kernel': ['rbf'],
    'C': [1.0, 3.0, 5.0],
    'gamma': [0.01, 0.1, 1.0]
    }

grid_search_SVR_2 = GridSearchCV(svm_reg, param_grid_2, cv=3,
                           scoring='neg_mean_squared_error',
                           refit= True, verbose=2, n_jobs=-1)
grid_search_SVR_2.fit(X_train, y_train) # Reentrena el mejor método obtenido

Fitting 3 folds for each of 9 candidates, totalling 27 fits


GridSearchCV(cv=3, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [1.0, 3.0, 5.0], 'gamma': [0.01, 0.1, 1.0],
                         'kernel': ['rbf']},
             scoring='neg_mean_squared_error', verbose=2)

PREGUNTA:
¿Qué RMSE alcanza la mejor solución?

In [37]:
#RESPUESTA
#Para el grid search variando el parámetro C y con kernel lineal:
np.sqrt(-grid_search_SVR.best_score_)

77.47723433757558

In [40]:
#Para el grid search variando los parámetros C, gamma y con kernel rbf:
np.sqrt(-grid_search_SVR_2.best_score_)

74.492830157436

PREGUNTA:
¿Cuál es la mejor combinación de hiperparámetros?

La mejor combinación de hiperparámetros son:

In [34]:
#RESPUESTA
#Para el grid search variando el parámetro C y con kernel lineal:
grid_search_SVR.best_params_

{'C': 3.0, 'kernel': 'linear'}

In [41]:
#Para el grid search variando los parámetros C, gamma y con kernel rbf:
grid_search_SVR_2.best_params_

{'C': 5.0, 'gamma': 1.0, 'kernel': 'rbf'}

PREGUNTA: Obtén los resultados de todos los hiperparámetros que ha probado. ¿Puedes extraer alguna conclusión sobre los parámetros usados en la mejor opción? Por ejemplo, si el parámetro elegido como mejor opción es el mayor (o menor) usado, podría valer la pena incluir mayores (o menores valores) para ese parámetro.

In [38]:
#RESPUESTA:
#Para el grid search variando el parámetro C y con kernel lineal:
cvres = grid_search_SVR.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

78.63275657220132 {'C': 1.0, 'kernel': 'linear'}
77.47723433757558 {'C': 3.0, 'kernel': 'linear'}


In [42]:
#Para el grid search variando los parámetros C, gamma y con kernel rbf:
cvres = grid_search_SVR_2.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

79.28733911714465 {'C': 1.0, 'gamma': 0.01, 'kernel': 'rbf'}
79.15053941483656 {'C': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}
78.18226028866891 {'C': 1.0, 'gamma': 1.0, 'kernel': 'rbf'}
79.25669271306619 {'C': 3.0, 'gamma': 0.01, 'kernel': 'rbf'}
78.85241322279813 {'C': 3.0, 'gamma': 0.1, 'kernel': 'rbf'}
76.15567642678245 {'C': 3.0, 'gamma': 1.0, 'kernel': 'rbf'}
79.22607611420248 {'C': 5.0, 'gamma': 0.01, 'kernel': 'rbf'}
78.63555169493115 {'C': 5.0, 'gamma': 0.1, 'kernel': 'rbf'}
74.492830157436 {'C': 5.0, 'gamma': 1.0, 'kernel': 'rbf'}


La conclusion que podemos obtener con los parámetros que hemos probado es que cuanto más altos mejores resultados obtenemos, tanto para C en solitario o combinado con Gamma como para Gamma.

Parece que el mejor "C" es al máximo testeado. Por ello, es conveniente repetir la búsqueda borrando los valores pequeños de C y cambiándolos por otros mayores.

EJERCICIO (Extra): Prueba a usar el kernel 'poly' con distintos valores para los parámetros 'C', 'gamma' y 'degree'; y el kernel 'sigmoid' con distintos valores para los parámetros 'C' y 'gamma'. Prueba también a cambiar el hiperparámetro 'epsilon' en todos los kernels ¿Consigues mejorar el resultado?

In [48]:
#SOLUCIÓN
%%time
#SOLUCIÓN
param_grid = {
    'kernel': ['poly'],
    'C': [3.0, 5.0, 8.0],
    'gamma': [0.1, 1.0, 1.5, 2.0],
    'degree': [3, 5, 8],
    'epsilon': [0.1, 1.0, 1.5]
    }

grid_search_SVR_poly = GridSearchCV(svm_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           refit= True, verbose=2, n_jobs=-1)
grid_search_SVR_poly.fit(X_train, y_train) # Reentrena el mejor método obtenido

Fitting 3 folds for each of 108 candidates, totalling 324 fits
CPU times: user 175 ms, sys: 8.21 ms, total: 183 ms
Wall time: 1.53 s


GridSearchCV(cv=3, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [3.0, 5.0, 8.0], 'degree': [3, 5, 8],
                         'epsilon': [0.1, 1.0, 1.5],
                         'gamma': [0.1, 1.0, 1.5, 2.0], 'kernel': ['poly']},
             scoring='neg_mean_squared_error', verbose=2)

In [49]:
#RESPUESTA El mejor modelo alcanza el siguiente score, (usando 3-fold cross validation):
np.sqrt(-grid_search_SVR_poly.best_score_)

79.28201215515566

In [50]:
grid_search_SVR_poly.best_params_

{'C': 8.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 2.0, 'kernel': 'poly'}

In [52]:
cvres = grid_search_SVR_poly.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

79.30267251616877 {'C': 3.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 0.1, 'kernel': 'poly'}
79.30170434322633 {'C': 3.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 1.0, 'kernel': 'poly'}
79.29940288382254 {'C': 3.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 1.5, 'kernel': 'poly'}
79.29492211698259 {'C': 3.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 2.0, 'kernel': 'poly'}
79.3524575387682 {'C': 3.0, 'degree': 3, 'epsilon': 1.0, 'gamma': 0.1, 'kernel': 'poly'}
79.35152248923097 {'C': 3.0, 'degree': 3, 'epsilon': 1.0, 'gamma': 1.0, 'kernel': 'poly'}
79.34929968097126 {'C': 3.0, 'degree': 3, 'epsilon': 1.0, 'gamma': 1.5, 'kernel': 'poly'}
79.34497169019114 {'C': 3.0, 'degree': 3, 'epsilon': 1.0, 'gamma': 2.0, 'kernel': 'poly'}
79.43758217101598 {'C': 3.0, 'degree': 3, 'epsilon': 1.5, 'gamma': 0.1, 'kernel': 'poly'}
79.43662533536938 {'C': 3.0, 'degree': 3, 'epsilon': 1.5, 'gamma': 1.0, 'kernel': 'poly'}
79.43435071744388 {'C': 3.0, 'degree': 3, 'epsilon': 1.5, 'gamma': 1.5, 'kernel': 'poly'}
79.42992187

In [64]:
#SOLUCIÓN
%%time
#SOLUCIÓN
param_grid = {
    'kernel': ['sigmoid'],
    'C': [3.0, 5.0, 8.0],
    'gamma': [0.1, 1.0, 1.5, 2.0],
    'epsilon': [0.1, 1.0, 1.5]
    }

grid_search_SVR_sigmoid = GridSearchCV(svm_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           refit= True, verbose=2, n_jobs=-1)
grid_search_SVR_sigmoid.fit(X_train, y_train) # Reentrena el mejor método obtenido

Fitting 3 folds for each of 36 candidates, totalling 108 fits
CPU times: user 91.6 ms, sys: 3.83 ms, total: 95.4 ms
Wall time: 1.07 s


GridSearchCV(cv=3, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [3.0, 5.0, 8.0], 'epsilon': [0.1, 1.0, 1.5],
                         'gamma': [0.1, 1.0, 1.5, 2.0], 'kernel': ['sigmoid']},
             scoring='neg_mean_squared_error', verbose=2)

In [65]:
#El mejor modelo alcanza el siguiente score, (usando 3-fold cross validation):
np.sqrt(-grid_search_SVR_sigmoid.best_score_)

72.0495110263543

In [66]:
grid_search_SVR_sigmoid.best_params_

{'C': 8.0, 'epsilon': 1.5, 'gamma': 2.0, 'kernel': 'sigmoid'}

In [67]:
cvres = grid_search_SVR_sigmoid.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

79.07329585309176 {'C': 3.0, 'epsilon': 0.1, 'gamma': 0.1, 'kernel': 'sigmoid'}
77.47740850143389 {'C': 3.0, 'epsilon': 0.1, 'gamma': 1.0, 'kernel': 'sigmoid'}
76.72878785657164 {'C': 3.0, 'epsilon': 0.1, 'gamma': 1.5, 'kernel': 'sigmoid'}
75.99200977091292 {'C': 3.0, 'epsilon': 0.1, 'gamma': 2.0, 'kernel': 'sigmoid'}
79.11983721411829 {'C': 3.0, 'epsilon': 1.0, 'gamma': 0.1, 'kernel': 'sigmoid'}
77.53313364330265 {'C': 3.0, 'epsilon': 1.0, 'gamma': 1.0, 'kernel': 'sigmoid'}
76.72051415670816 {'C': 3.0, 'epsilon': 1.0, 'gamma': 1.5, 'kernel': 'sigmoid'}
76.00448453082797 {'C': 3.0, 'epsilon': 1.0, 'gamma': 2.0, 'kernel': 'sigmoid'}
79.19822608219674 {'C': 3.0, 'epsilon': 1.5, 'gamma': 0.1, 'kernel': 'sigmoid'}
77.61998769289636 {'C': 3.0, 'epsilon': 1.5, 'gamma': 1.0, 'kernel': 'sigmoid'}
76.7742950349117 {'C': 3.0, 'epsilon': 1.5, 'gamma': 1.5, 'kernel': 'sigmoid'}
76.09623867032974 {'C': 3.0, 'epsilon': 1.5, 'gamma': 2.0, 'kernel': 'sigmoid'}
78.9213158098824 {'C': 5.0, 'epsilon': 0.

Nuevamente las conclusiones son las mismas para C, gamma y epsilon, cuanto mayores, mejores resultados obtenemos. No ocurre lo mismo con el grado/degree ya que es un parámetro que funciona diferente y tiene que ver con la naturaleza del modelo cuando es polinómico. Se trata del grado del polinomio utilizado en la regresión. No necesariamente cuanto más grande sea obtendremos un mejor resultado.

# Curva de validación

Determina los resultados del modelo cuando varía el valor de un parámetro. Para ello realiza validación cruzada sobre el conjunto de entrenamiento y permite representar los resultados (sobre entrenamiento y validación) para detectar, por ejemplo, si variar un determinado valor produce sobreentrenamiento.

Es similar a una búsqueda en red pero con un único parámetro.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import validation_curve

#Lista de parámetros
param_range = list(range(2,10,2))

#Curva de validación
train_scores, validation_scores = validation_curve(
    forest_reg, X_train, y_train, param_name="max_depth", param_range=param_range,
    scoring='neg_root_mean_squared_error', cv=3, n_jobs=1)

#Pintamos la curva
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
validation_scores_mean = np.mean(validation_scores, axis=1)
validation_scores_std = np.std(validation_scores, axis=1)

plt.title("Curva de validation para forest regressor")
plt.xlabel(r"max_depth")
plt.ylabel("Score")
lw = 2
plt.plot(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.plot(param_range, validation_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, validation_scores_mean - validation_scores_std,
                 validation_scores_mean + validation_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

Se observa que variar el parámetro "máxima profundidad" mejora el entrenamiento, pero produce sobre-entrenamiento. De hecho aumenta ligeramente el resultado hasta 4 características y después disminuye.

## Ejercicios Curva de validación

EJERCICIO: Obtén la curva de validación para método de random forest anterior para el parámetro 'n_estimators' entre 10 y 200, con incrementos de 10. Da una interpretación a la gráfica obtenida.

In [ ]:
# Solución
%%time
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import validation_curve



EJERCICIO (Extra): Obtén la curva de validación para método de SVR anterior para el parámetro 'C' entre 0.1 y 40, con incrementos de 0.5. Da una interpretación a la gráfica obtenida.

In [ ]:
%%time
param_range = [x/10 for x in range(1,400,5)]  #Lista de parámetros

# Solución



# Random Search

La búsqueda aleatoria evalúa un número determinado de combinaciones seleccionando un valor aleatorio para cada hiperparámetro en cada iteración (en lugar de fijar los valores en los que buscar). Este enfoque tiene dos beneficios principales:

- Si se deja que la búsqueda aleatoria se realice para, digamos, 100 iteraciones, este enfoque explorará 100 valores diferentes para cada hiperparámetro (en lugar de sólo unos pocos valores por hiperparámetro con el enfoque de búsqueda en cuadrícula).
- Se tiene más control sobre la cantidad de computación que desea asignar a la búsqueda de hiperparámetros, simplemente estableciendo el número de iteraciones.

In [ ]:
%%time

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# ver https://docs.scipy.org/doc/scipy/reference/stats.html
# para documentación sobre posibles funciones de distribución de probalidad.

param_distribs = {
        'n_estimators': randint(low=30, high=300), #A uniform discrete random variable.
        'max_features': randint(low=2, high=10)
    }

forest_reg = RandomForestRegressor(random_state=42)

rnd_search_RF = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=3, scoring='neg_mean_squared_error', random_state=42, verbose=2, n_jobs=-1)

rnd_search_RF.fit(X_train, y_train)

negative_mse = rnd_search_RF.best_score_
rmse = np.sqrt(-negative_mse)
print("rmse del mejor modelo", rmse)

In [ ]:
rnd_search_RF.best_params_



Las opciones que ha probado son:

In [ ]:
cvres = rnd_search_RF.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

Se obtiene un error similar (es un poco peor) probando 10 candidatos.

Vamos a evaluar los mejores modelos encontrados con grid search y con random search sobre el conjunto de test.


In [ ]:
from sklearn.metrics import mean_squared_error

final_model_grid_RF = grid_search_RF.best_estimator_

final_predictions_grid_RF = final_model_grid_RF.predict(X_test)

final_mse_grid = mean_squared_error(y_test, final_predictions_grid_RF)
final_rmse_grid = np.sqrt(final_mse_grid)
print('Mejor resultado con grid search sobre el conjunto de test',final_rmse_grid)

In [ ]:
final_model_rnd_RF = rnd_search_RF.best_estimator_
final_predictions_rnd_RF = final_model_rnd_RF.predict(X_test)

final_mse_rnd = mean_squared_error(y_test, final_predictions_rnd_RF)
final_rmse_rnd = np.sqrt(final_mse_rnd)
print('Mejor resultado con random search sobre el conjunto de test', final_rmse_rnd)

El rendimiento que obtenemos es un poco peor en este caso, mirando menos alternativas. Debemos resistirnos a continuar haciendo ajuste de los hiperparámetros una vez testeado nuestro modelo en el conjunto de test, ya que las mejoras podrían no generalizarse a un nuevo conjunto de datos.

## Ejercicios Random Search.

EJERCICIO: Sobre el Ejercicio para Random Search para modelo Support Vector Machine Regressor cambiar la búsqueda GridSearchCV por una búsqueda RandomizedSearchCV.

Como distribuciones de búsqueda usar:

- Un lista ['linear', 'rbf'] para el parámetro 'kernel'
- loguniform(1, 20) para el parámetro 'C'
- expon(scale=1.0) para el parámetro 'gamma'

Utiliza un máximo de 10 modelos y cross validation con 3 folds.

Podemos encontrar en el siguiente [enlace](https://docs.scipy.org/doc/scipy/reference/stats.html) documentación para las funciones de distribución de probabilidad `expon()` y `loguniform()`.

In [ ]:
#Respuesta
%%time



PREGUNTA:
¿Cuál es el mejor RMSE que se obtiene?¿Cuáles son los hiperparámetros con los que ha probado?

In [ ]:
#RESPUESTA: El mejor modelo obtiene un RMSE:


In [ ]:
#RESPUESTA: Ha probado los siguientes hiperparámetros:


PREGUNTA: ¿Se supera el resultado obtenido con Grid Search? ¿Qué parámetros obtienen el mejor resultado?

In [ ]:
#RESPUESTA:


En esta ocasión se ha encontrado un conjunto de hiperparámetros adecuados para el kernel RBF. La búsqueda aleatoria tiende a encontrar mejores hiperparámetros que la búsqueda en cuadrícula en la misma cantidad de tiempo.

EJERCICIO (Extra): Fijarse que hemos decidido usar una escala con distribución logarítmica uniforme para C y exponencial para 'gamma'.
En el siguiente gráfico representamos estas distribuciones para 1000 elementos. Explica brevemente qué repercusión tiene la anterior decisión en los valores elegidos para estas variables. Atendiendo al valor óptimo que hemos obtenido para el parámetro C, ¿crees que ha sido una buena decisión elegir la distribución logarítmica para C?

In [ ]:
loguniform_distrib = loguniform(1, 20)
samples = loguniform_distrib.rvs(10000, random_state=42)
plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.title("log unifom distribution (2, 20)")
plt.hist(samples, bins=50)

expon_distrib = expon(scale=1.0)
samples = expon_distrib.rvs(10000, random_state=42)
plt.subplot(122)
plt.title("expon distribution (scale=1.0)")
plt.hist(samples, bins=50)
plt.show()

OPINIÓN:


EJERCICIO (Extra): A la vista del resultado obtenido para C, ¿cambiarías de distribución estadística para este parámetro? Prueba a cambiar simplemente la escala a loguniform(1,200) y comprueba cuáles son los mejores valores para los hiperparámetros así como los resultados que obtienes tanto en el conjunto de validación como en el conjunto de test.

In [ ]:
#RESPUESTA
# Nota: gamma se ignora cuando el kernel es "linear"


In [ ]:
negative_mse = rnd_search_svm_reg.best_score_
rmse = np.sqrt(-negative_mse)
rmse

In [ ]:
rnd_search_svm_reg.best_params_

In [ ]:
final_model_rnd_reg = rnd_search_svm_reg.best_estimator_
final_predictions_rnd_reg = final_model_rnd_reg.predict(X_test)

final_mse_rnd = mean_squared_error(y_test, final_predictions_rnd_reg)
final_rmse_rnd = np.sqrt(final_mse_rnd)
print('Mejor resultado con random search sobre el conjunto de test', final_rmse_rnd)

# Optimización Bayesiana

Es un ejemplo de modelo secuencial que permiten utilizar los resultados de una iteración para mejorar el modelo en la siguiente. En particular permite construir un modelo de probabilidad bayesiano de la función objetivo y usarlo para seleccionar los parámetros más prometedores en la función objetivo real.



Existen varias librerías en python que han implementado métodos de optimización bayesiana. Entre ellas se encuentran [scikit-optimice](https://scikit-optimize.github.io/stable/) y [hyperopt](https://github.com/hyperopt/hyperopt).

Usamos a continuación el método BayesSearchCV de la librería scikit-optimize

In [ ]:
! pip install scikit-optimize

In [ ]:
from skopt import BayesSearchCV

In [ ]:
%%time
import matplotlib.pyplot as plt
import numpy as np

from skopt.space import Real, Categorical, Integer #Permite definir el tipo
param_distribs_exp = {  #permite indicar la distribución de los parámetros
        'n_estimators': Integer(3, 300, 'uniform'), #Permite Integer, Categorical o Real
        'max_features': Integer(3, 8, 'uniform'), #Permite 'uniform' o 'log-uniform'
    }


forest_reg = RandomForestRegressor(random_state=42)

bayes_search_RF = BayesSearchCV(
    forest_reg,
    param_distribs_exp,
    n_iter=20,
    cv=3, scoring='neg_mean_squared_error', random_state=42,
    n_jobs=-1,
    optimizer_kwargs={'base_estimator': 'GP'} # 'GP': Gaussean Process,'RF': Random Forest regressor,'ET': Extra Trees regressor, 'GBRT': gradient boosted trees
  )
bayes_search_RF.fit(X_train, y_train)

print(bayes_search_RF.best_params_)
mean_score = bayes_search_RF.cv_results_['mean_test_score'][bayes_search_RF.best_index_]
print("score", np.sqrt(-mean_score))



Obtiene un resultado similar.

In [ ]:
cvres_bayes = bayes_search_RF.cv_results_
for mean_score, params in zip(cvres_bayes["mean_test_score"], cvres_bayes["params"]):
    print(np.sqrt(-mean_score), params)

Los resultados que ha ido obteniendo son:

In [ ]:
x = np.arange(0, 20)
plt.plot(x, [np.sqrt(-x) for x in cvres_bayes["mean_test_score"]])
plt.show()

Probamos en el conjunto de test.

In [ ]:
from sklearn.metrics import mean_squared_error

final_model_bayes = bayes_search_RF.best_estimator_

final_predictions_bayes = final_model_bayes.predict(X_test)

final_mse_bayes = mean_squared_error(y_test, final_predictions_bayes)
final_rmse_bayes = np.sqrt(final_mse_bayes)
final_rmse_bayes




## Ejercicios Optimización bayesiana

EJERCICIO: Utiliza para el SVR() del ejemplo anterior el método BayesSearchCV. Usa el espacio de búsqueda que se propone.

Utiliza el algoritmo 'GBRT' con 10 iteraciones.

¿Cuál es el mejor resultado que obtienes?¿Con qué configuración de parámetros?

In [ ]:
from skopt.space import Real, Categorical, Integer
param_distribs_exp = {
        'kernel':  Categorical(['linear', 'rbf']),
        'C': Real(1, 20, 'log-uniform'),
        'gamma': Real(1e-6, 5, 'log-uniform'),
    }


svm_reg = SVR()

In [ ]:
# Solución
%%time



EJERCICIO: Sobre el mismo espacio de búsqueda anterior y usando el algoritmo 'GBRT' y 50 iteraciones, obtén el resultado sobre el conjunto de entrenamiento de los 50 modelos. ¿Se observa una mejoría en la predicción a medida que se van obteniendo modelos?¿Por qué? Muestra la evolución de los resultados en una gráfica.

In [ ]:
# Solución
%%time




EJERCICIO: Obtén sobre el conjunto de test los resultados con el mejor método obtenido. ¿Mejora los métodos aplicados previamente sobre el SVR?

In [ ]:
#Solución


EJERCICIO (Extra). Prueba a cambiar los kernels, añadir hiperparámetros (por ejemplo 'epsilon' o 'degree') y/o aumentar el número de iteraciones. Comprueba si se obtiene alguna mejora sobre el conjunto de validación. Prueba a cambiar el estimador base a 'GP'.  Finalmente, comprueba los resultados de la mejor opción sobre el conjunto de test.

In [ ]:
from skopt.space import Real, Categorical, Integer
param_distribs_exp = {
        'kernel':  Categorical(['linear', 'rbf', 'poly']),
        'C': Real(1, 20, 'log-uniform'),
        'gamma': Real(1e-6, 5, 'log-uniform'),
        'epsilon': Real(1e-6, 5, 'log-uniform'),
        'degree': [2, 3, 4]
    }


svm_reg = SVR()

# Solución



In [ ]:
# Mejor opción sobre el conjunto de test
#Solución
final_model_svm_reg = bayes_search_svm_reg.best_estimator_
final_predictions_svm_reg = final_model_svm_reg.predict(X_test)

final_mse_svm_reg = mean_squared_error(y_test, final_predictions_svm_reg)
final_rmse_svm_reg = np.sqrt(final_mse_svm_reg)
final_rmse_svm_reg

# División de recursos

En la última versión de scikit-learn han implementado algoritmos basados en la división de recursos denomindos  HalvingGridSearchCV y HalvingRandomSearchCV, que pueden ser más rápidos encontrando la mejor conbinación de hiperparámetros.

Este tipo de algoritmos para el ajuste de hiperparámetros se basan en el supuesto de que si una configuración está destinada a ser la mejor después de un gran número de iteraciones, es más probable que funcione en la mitad superior de las configuraciones después de un pequeño número de iteraciones.

Así no se hace hincapié en el rendimiento absoluto de un algoritmo, sino más bien en su rendimiento relativo en comparación con muchas alternativas entrenadas para el mismo número de iteraciones.

Existen otras librerías fuera de scikit-learn que implementan otros algoritmos de división de recursos como la librería [hyperband](://thuijskens.github.io/scikit-hyperband/docs/hyperband.html).

Cargamos la librería HalvingGridSearchCV

In [ ]:
from sklearn.experimental import enable_halving_search_cv  # Está en fase experimental
from sklearn.model_selection import HalvingGridSearchCV

Cargamos la DB diabetes de nuevo y repartimos los conjuntos train y test como antes.

In [ ]:
from sklearn import datasets

diabetes_BD = datasets.load_diabetes()
print(diabetes_BD.DESCR)


In [ ]:
diabetes=diabetes_BD["data"] # Datos de las casas a usar
feature_names=diabetes_BD["feature_names"] # Nombre de las variables
diabetes_target=diabetes_BD["target"] # Dato a inferir. Precio medio de las casas

In [ ]:
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split(diabetes, diabetes_target, test_size=0.2, random_state=42)

Vamos a aplicar el algoritmo a un Random Forest.

Por defecto, en cada iteración existe un recurso (controlado por el parámetro *resource*) que se va incrementando en cada iteración a los elementos que mejor funcionen en dicha iteración. Por defecto, el recurso es el número de muestras de la fase de entrenamiento. Puede elegirse cualquier parámetro que acepte un número positivo (por ejemplo, 'n_iterations').

Cuando se aplica a Random Forest, el parámetro que se ajusta en cada estimación suele ser el número de árboles ('n_estimators'). En este ejemplo, coincide que también es uno de los parámetros a combinar.

Añadimos algún parámetro más a probar como *max_depth* para aumentar el espacio de búsqueda. Recordar que n_estimators lo usaremos para ir repartiendo los recursos.


In [ ]:
# Distributions or lists of parameters to try. Distributions must provide a rvs method for sampling (such as those from scipy.stats.distributions). If a list is given, it is sampled uniformly.
param_grid = {
     #'n_estimators': [100, 300, 900],
     'max_features': [2, 4, 6, 8],
      'max_depth' : [3,5, 7, 9, None],
          }

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42)
forest_reg

max_resources y min_resources marcan el número máximo de árboles que se asinganarán a una combinación de hiperparámetros, y factor la reducción de configuraciones en cada iteración. Cada vez que se reduzca en factor las configuraciones se multiplicará por factor los recursos asignados a las supervivientes.

In [ ]:
%%time

halv_grid_search_RF = HalvingGridSearchCV(forest_reg,
                           param_grid,
                           resource='n_estimators', #Recursos que se asignarán en cada estimacion. En Random Forest tipicamente el número de árboles
                           verbose =1,
                           max_resources=1000, #Numero de árboles máximo en una iteración
                           min_resources=100, #Numero de árboles mínimo en una iteración
                           factor=3, #Porcentaje de configuraciones que sobrevivirán en cada iteración, en este caso se reducirá a un tercio
                           cv=3,
                           scoring='neg_mean_squared_error',
                           random_state = 42)

halv_grid_search_RF.fit(X_train, y_train)

Obtiene una solución en unos 30 segundos. Haciendo 3 itieraciones. La primera mira todas los candidatos (20) con 100 árboles. En la segunda se queda con un tercio (factor = 3) de los mejores candidatos (7) tripicando los recursos (300 árboles). Y finalmente en la última prueba con los 3 mejores candidatos con 900 árboles.

El resultado obtenido es:

In [ ]:
import numpy as np

negative_mse = halv_grid_search_RF.best_score_
rmse = np.sqrt(-negative_mse)
rmse

Las combinaciones que ha probado son:

In [ ]:
cvres = halv_grid_search_RF.cv_results_
print(len(cvres["params"]))
i=0
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    i+=1
    print(i, np.sqrt(-mean_score), params)

Finalmente, probamos el método obtenido sobre el conjunto de test.

In [ ]:
from sklearn.metrics import mean_squared_error

final_model_hyb = halv_grid_search_RF.best_estimator_
final_predictions_hyb = final_model_hyb.predict(X_test)

final_mse_hyb = mean_squared_error(y_test, final_predictions_hyb)
final_rmse_hyb = np.sqrt(final_mse_hyb)
final_rmse_hyb

La alternativa con grid search es probar con todos los recursos (900 árboles) para todos los candidatos.

La probamos.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Distributions or lists of parameters to try. Distributions must provide a rvs method for sampling (such as those from scipy.stats.distributions). If a list is given, it is sampled uniformly.
param_grid = {
     'n_estimators': [900],
     'max_features': [2, 4, 6, 8],
      'max_depth' : [3,5, 7, 9, None],
          }



In [ ]:
%%time
grid_search_RF = GridSearchCV(forest_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           refit= True, verbose=2, n_jobs=-1)
grid_search_RF.fit(X_train, y_train)

Encuentra una solución en más de 1 minuto.

Las mejores soluciones encontradas son en este caso la misma.

In [ ]:
grid_search_RF.best_params_

In [ ]:
halv_grid_search_RF.best_params_

## Ejercicios División de recursos

EJERCICIO: Emplea HalvingRandomSearchCV sobre el ejercicio SVR.

Utiliza el mismo espacio para los parámetros que en el primer ejercicio del bloque de ejercicios del Random Search.

Como parámetro de división de recursos sobre SVR usa el parámetro 'n_samples' (opción por defecto), con número de recursos mínima 'smallest" y máxima 'auto' (total de la muestra), total de candidatos 'n_candidates' a explorar de 30 y el factor de división de recursos a 2.

In [ ]:
%%time

from sklearn.model_selection import HalvingRandomSearchCV
import numpy as np
from sklearn.metrics import mean_squared_error

from sklearn.svm import SVR
from scipy.stats import expon, loguniform

param_grid_SVR = {
        'kernel': ['linear', 'rbf'],
        'C': loguniform(1, 20), #A loguniform or reciprocal continuous random variable.
        'gamma': expon(scale=1.0), #An exponential continuous random variable.
            }

svm_reg = SVR()

# Solución


halv_random_search_SVR = HalvingRandomSearchCV(svm_reg,
                           param_grid_SVR,
                           resource='n_samples', #Recursos que se asignarán en cada estimacion. En Random Forest tipicamente el número de árboles
                           verbose =1,
                           max_resources='auto', #Numero de árboles máximo en una iteración
                           min_resources='smallest', #Numero de árboles mínimo en una iteración
                           factor=2, #Porcentaje de configuraciones que sobrevivirán en cada iteración, en este caso se reducirá a un tercio
                           n_candidates=30,
                           cv=3,
                           scoring='neg_mean_squared_error',
                           random_state = 42)

halv_random_search_SVR.fit(X_train, y_train)



PREGUNTA: ¿Cuáles son los mejores valores para los hiperparámetros? ¿Cuál es el mejor resultado?¿Cuántas opciones ha probado?

In [ ]:
#Solución

halv_random_search_SVR.best_params_

Ha probado 138 veces ( 30 + 10 + 4 + 2 ) * 3


In [ ]:
# El mejor resultado:
negative_mse = halv_random_search_SVR.best_score_
rmse = np.sqrt(-negative_mse)
rmse

EJERCICIO: Obtén sobre el conjunto de test los resultados con el mejor método obtenido. ¿Mejora los métodos aplicados previamente sobre el SVR? ¿Mejora el tiempo de ejecución?

In [ ]:
from sklearn.metrics import mean_squared_error

final_model_halvRandom = halv_random_search_SVR.best_estimator_
final_predictions_halvRandom = final_model_halvRandom.predict(X_test)

final_mse_hRand = mean_squared_error(y_test, final_predictions_halvRandom)
final_rmse_hRand = np.sqrt(final_mse_hRand)
final_rmse_hRand

In [ ]:
grid_search_SVR = GridSearchCV(svm_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           refit= True, verbose=2, n_jobs=-1)

In [ ]:
# Solución

param_grid_SVR = {
        'kernel': ['linear', 'rbf'],
        'C': [loguniform(1, 20)], #A loguniform or reciprocal continuous random variable.
        'gamma': [expon(scale=1.0)], #An exponential continuous random variable.
            }

halv_grid_search_SVR = HalvingGridSearchCV(svm_reg,
                           param_grid_SVR,
                           resource='n_samples', #Recursos que se asignarán en cada estimacion. En Random Forest tipicamente el número de árboles
                           verbose =1,
                           max_resources='auto', #Numero de árboles máximo en una iteración
                           min_resources='smallest', #Numero de árboles mínimo en una iteración
                           factor=2, #Porcentaje de configuraciones que sobrevivirán en cada iteración, en este caso se reducirá a un tercio
                           cv=3,
                           scoring='neg_mean_squared_error',
                           random_state = 42)

halv_grid_search_SVR.fit(X_train, y_train)



EJERCICIO (Extra): En redes neuronales un parámetro típico en el que interviene la división de recursos es en el número de itereciones a entrenar la red 'max_iter'. Aplica HalvingGridSearchCV a la siguiente red neuronal aplicada a regresiones [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html). Incluye diferentes las opciones para los parámetros 'solver', 'activation', 'alpha' y 'momentum' que se proporcionan (puedes incluir otras si quieres). Elige un factor de reducción 3. El número mínimo de recursos será 2000 y el máximo 20000. Obten la mejor opción sobre el conjunto de validación y test. Registra el tiempo que se emplea.

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

NN_reg = MLPRegressor(solver='sgd', alpha=0.01, momentum=0.1, activation='relu',
                    hidden_layer_sizes= (5, 2), random_state=42,  max_iter=10000)

print(NN_reg.fit(X_train, y_train))

scores = cross_val_score(NN_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=3)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

NN_reg = MLPRegressor(solver='adam', alpha=0.01, momentum=0.1, activation='relu',
                    hidden_layer_sizes= (5, 2), random_state=42,  max_iter=2000)

print(NN_reg.fit(X_train, y_train))

scores = cross_val_score(NN_reg, X_train, y_train, scoring="neg_mean_squared_error", cv=3)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

En particular

In [ ]:

NN_reg_params = {
    'solver': ['sgd', 'adam','lbfgs'],
    'activation': ['relu','tanh', 'logistic'],
    'alpha': [0.01, 0.1, 0.5, 0.9],
    'momentum': [0.1, 0.5, 0.9],
}


In [ ]:
# Solución
# %%time



EJERCICIO (Extra): Añade a los valores de los hiperparámetros utilizados en el ejercicio anterior el hiperparámetro con valores 'max_iter': [18000]. Estos son las iteraciones que se usaron en la división de recursos a los métodos ganadores. Aplica a ese conjunto de hiperparámetros el algoritmo GridSearchCV sobre la red neuronal anterior. ¿Se obtiene la misma solución?¿Se obtiene en el mismo tiempo? Explica los resultados obtenidos.

In [ ]:
# Solución
# %%time



# Métodos de optimización basados en la naturaleza

Compara distintos tipos de ajuste de hiperparámetros inspirados por la naturaleza.

Estos algoritmos tratan de imitar el comportamiento de algunos seres vivos en rutinas algorítmicas de optimización.

- Un **individuo** es una combinación de parámetros a ser entrenados durante un ciclo de optimización. Los resultados de cada individuo son almacenados entre ciclos.
- Una **población** es un grupo de individuos en una generación/ciclo de optimización.

Los algoritmos que siguen están modificados para que sea razonable de ejecutar en tiempo. Pero se pierde parte de la gracia ya que son más rápidos cuando existen muchos hiperparámetros a buscar.

ATENCIÓN: Necesita reiniciar el entorno de ejecución.

In [ ]:
#! instalamos la librería sklearn_nature_inspired_algorithms
!pip install sklearn_nature_inspired_algorithms
#Restart kernel after installation
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
from sklearn_nature_inspired_algorithms.model_selection.nature_inspired_search_cv import NatureInspiredSearchCV
from sklearn_nature_inspired_algorithms.helpers import score_by_generation_lineplot

Intentamos de nuevo hacer la búsqueda de hiperparámetros para un random forest, sobre el conjunto de datos que estamos usando.

In [ ]:
from sklearn import datasets

diabetes_BD = datasets.load_diabetes()

In [ ]:
diabetes=diabetes_BD["data"] # Datos de las casas a usar
feature_names=diabetes_BD["feature_names"] # Nombre de las variables
diabetes_target=diabetes_BD["target"] # Dato a inferir. Precio medio de las casas

In [ ]:
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split(diabetes, diabetes_target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42)
forest_reg

In [ ]:
param_grid = {
    # 12 (3×4) combinaciones de hiperparámetros
    'n_estimators': [30, 100, 300],
    'max_features': [3, 6, 8, 'auto']
    }


La búsqueda se realiza a través de [NatureInspiredSearchCV](https://sklearn-nature-inspired-algorithms.readthedocs.io/en/stable/introduction/nature-inspired-search-cv.html). El conjunto de algoritmos implementados puede consultarse [aquí](https://sklearn-nature-inspired-algorithms.readthedocs.io/en/stable/introduction/algorithms.html).

In [ ]:
%%time

nia_search_RF = NatureInspiredSearchCV(
    forest_reg,
    param_grid,
    cv=3,
    verbose=2,
    algorithm='ba', #Bat Algorithm
    population_size=5, #Número de estimadores en la población. Cuando la población es pequeña la búsqueda será más rápida pero puede que no se llegue a obtener el mejor resultado (que se alcanzará cuando el tamaño de la población es similar a la posible combianción de parámetros).
    max_n_gen=100, #Número máximo de generaciones a ser optimizadas.
    max_stagnating_gen=5, #Si el resultado permanece invariable durante este número de generaciones la optimización se detiene.
    n_jobs=-1,
    scoring="neg_mean_squared_error",
    random_state=42)

nia_search_RF.fit(X_train, y_train)
print(nia_search_RF)

In [ ]:
nia_search_RF.best_params_

El mejor resultado obtenido es el mismo que con grid search:

In [ ]:
import numpy as np
negative_mse = nia_search_RF.best_score_
rmse = np.sqrt(-negative_mse)
rmse

Si probamos con grid search.

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

grid_search_RF = GridSearchCV(forest_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose=2, n_jobs=-1)
grid_search_RF.fit(X_train, y_train)

Obtenemos el mismo en un tiempo similar.

Con pocas opciones de hiperparámetros no se nota demasiado.
Si aumentamos el número de opciones.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

param_grid ={
    'n_estimators': [100, 300, 500],
    'max_features': [3, 6, 8, 'auto'],
    'max_depth': [5, 10],
    'min_samples_leaf': [1,2,4],
    "bootstrap":[True,False]
}
forest_reg = RandomForestRegressor(random_state=42)
# entrenamiento para 3 folds, sobre un total de (3*4*2*3*2)*3=432 rondas de entrenamiento

In [ ]:
%%time

grid_search_RF = GridSearchCV(forest_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose=2, n_jobs=-1)
grid_search_RF.fit(X_train, y_train)

Tarda sobre 3 minutos.

In [ ]:
grid_search_RF.best_params_

El mejor resultado obtenido es:

In [ ]:
negative_mse = grid_search_RF.best_score_
rmse = np.sqrt(-negative_mse)
rmse

Sobre el conjunto de test.

In [ ]:
from sklearn.metrics import mean_squared_error

final_model_grid = grid_search_RF.best_estimator_
final_predictions_grid = final_model_grid.predict(X_test)

final_mse_grid = mean_squared_error(y_test, final_predictions_grid)
final_rmse_grid = np.sqrt(final_mse_grid)
final_rmse_grid

Obtiene un error de algo más 53 puntos.
Probamos con el algoritmo basado en la naturaleza.

In [ ]:
%%time

nia_search_RF = NatureInspiredSearchCV(
    forest_reg,
    param_grid,
    cv=3,
    verbose=0,
    algorithm='ba', #ba, Bat Algorithm; hba, Hybrid Bat Algorithm; fa, Firefly Algorithm; hsaba, Hybrid Self Adaptive Bat Algorithm; gwo, Grey Wolf Optimizer
    population_size=10, #50
    max_n_gen=20, #100
    max_stagnating_gen=5,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    random_state=42)

nia_search_RF.fit(X_train, y_train)

Encuentra el mismo en menos de 2 minutos.

In [ ]:
nia_search_RF.best_params_

In [ ]:
negative_mse = nia_search_RF.best_score_
rmse = np.sqrt(-negative_mse)
rmse

Probando solo 82 opciones de las 144. En ejemplos documentados donde el número de opciones es mayor, la mejoría en tiempo también es más considerable.

In [ ]:
cvres = nia_search_RF.cv_results_
print(len(cvres["params"]))
i=0
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    i+=1
    print(i, np.sqrt(-mean_score), params)

## Ejercicios Algoritmos basados en la naturaleza

EJERCICIO: Aplica un algoritmo basado en la naturaleza para tratar de ajustar los hiperparámetros en el caso del SVR. Prueba incialmente con la siguiente cuadrícula de parámetros


In [ ]:
from sklearn.svm import SVR
from scipy.stats import expon, loguniform

param_grid_SVR =  {'kernel': ['rbf'], 'C': [1.0, 3.0, 5.0], 'gamma': [0.01, 0.1, 1.0]}

svm_reg = SVR()

In [ ]:
# Solución
# %%time


EJERCICIO: ¿Cuáles son los mejores valores para los hiperparámetros? ¿Cuál es el mejor resultado?¿Cuántas opciones ha probado?

In [ ]:
#Solución


EJERCICIO (Extra): Prueba a aumentar el número de hiperparámetros para SVR y el número de valores para estos hiperparámetros. Prueba a usar distintos algoritmos basados en la naturaleza de entre los disponibles en la librería. Documenta brevemente el tiempo que emplean y el resultados obtenido.

# Algoritmos genéticos

Los algorítmos genéticos están basados en la teoría de la evolución, en el que los individuos con mejor capacidad y adaptabilidad al medio son los que tienen más posibilidades de sobrevivir y pasar sus características a la siguiente generación.

La aplicación de algorítmos genéticos al ajuste de hiperparámetros considera cada hiperparámetro como una caráterística o cromosoma, y los valores que pueden tomar dichos cromosomas pueden considerarse como genes. Así, los genes de los cromosomas que producen un buen rendimiento son los candidatos a pasar a la siguiente generación (selección). No obstante, se produce una proporción de intercambio de cromosomas y genes entre los padres en la siguiente generación (cruce) lo que genera nuevas combinaciones de cromosomas. También se puede producir una alteración aleatoria en uno o varios genenes (mutación).  Los cruces y mutaciones permiten que las nuevas generaciones tengan características diferentes de los padres y reduzcan la posibilidad de perder características interesantes.

Existen distintintas librerías que implementan algoritmos genéticos. Ejemplos son [sklearn-deap](https://github.com/rsteca/sklearn-deap) o [tpot](https://github.com/EpistasisLab/tpot).

Vamos a usar la librería [sklearn-deap](https://github.com/rsteca/sklearn-deap)


In [ ]:
!pip install scikit-learn==1.0.2
#Restart kernel after installation
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
!pip install sklearn-deap

Cargamos la DB diabetes de nuevo y repartimos los conjuntos train y test como antes.

La volvemos a probar sobre el random forest.

In [ ]:
from sklearn import datasets

diabetes_BD = datasets.load_diabetes()
print(diabetes_BD.DESCR)


In [ ]:
diabetes=diabetes_BD["data"] # Datos de las casas a usar
feature_names=diabetes_BD["feature_names"] # Nombre de las variables
diabetes_target=diabetes_BD["target"] # Dato a inferir. Precio medio de las casas

In [ ]:
from sklearn.model_selection import train_test_split
X_train,  X_test, y_train, y_test = train_test_split(diabetes, diabetes_target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42)
forest_reg

In [ ]:
#Random Forest
!pip install evolutionary_search


In [ ]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from scipy.stats import randint as sp_randint
# Define the hyperparameter configuration space

param_grid = {
    # 12 (3×4) combinaciones de hiperparámetros
    'n_estimators': [30, 100, 300],
    'max_features': [3, 6, 8, 'auto']
    }

En este caso la clase se llama EvolutionaryAlgorithmSearchCV

In [ ]:
%%time

ga_RF = EvolutionaryAlgorithmSearchCV(estimator=forest_reg,
                                   params=param_grid,
                                   scoring="neg_root_mean_squared_error", #Podemos poner RMSE
                                   cv=3,
                                   verbose=1,
                                   population_size=20, #Tamaño de la población
                                   gene_mutation_prob=0.10, # Probabilidad de mutación de un gen en un cromosoma
                                   gene_crossover_prob=0.5, # La probabilidad de intercambio de genes entre dos cromosomas
                                   generations_number=5,  # Número de generaciones
                                   n_jobs=1)
ga_RF.fit(X_train, y_train)

Encuentra la mejor opción.

In [ ]:
cvres = ga_RF.cv_results_
print(len(cvres["params"]))
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(-mean_score, params)

Si probamos con más parámetros.

In [ ]:
param_grid ={
    'n_estimators': [10, 20, 30, 100],
    'max_features': ['sqrt',0.5],
    'max_depth': [2, 5, 10, 15],
    'min_samples_leaf': [1,2,4],
    "bootstrap":[True,False]
    }

In [ ]:
%%time

ga_RF = EvolutionaryAlgorithmSearchCV(estimator=forest_reg,
                                   params=param_grid,
                                   scoring="neg_root_mean_squared_error", #Podemos poner RMSE
                                   cv=3,
                                   verbose=1,
                                   population_size=20, #Tamaño de la población
                                   gene_mutation_prob=0.10, # Probabilidad de mutación de un gen en un cromosoma
                                   gene_crossover_prob=0.5, #La probabilidad de intercambio de genes entre dos cromosomas
                                   generations_number=5,  #Número de generaciones
                                   n_jobs=1)
ga_RF.fit(X_train, y_train)

In [ ]:
cvres = ga_RF.cv_results_
print(len(cvres["params"]))
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(-mean_score, params)

## Ejercicios Algoritmos genéticos

EJERCICIO: Aplica algoritmos genéticos en la opción de SVM, para el siguiente conjunto de hiperparámetros.

In [ ]:
from sklearn.svm import SVR

In [ ]:
#SVM
%%time
import numpy as np

from evolutionary_search import EvolutionaryAlgorithmSearchCV
rf_params = {
    'C': np.random.uniform(0,50,1000),
    "kernel":['poly','rbf','sigmoid'],
    'gamma': np.random.uniform(0,1,100),
}
clf = SVR()

In [ ]:
#Solución

EJERCICIO: ¿Cuáles son los mejores valores para los hiperparámetros? ¿Cuál es el mejor resultado?¿Cuántas opciones ha probado?

In [ ]:
#Solución


EJERCICIO (Extra): Prueba a aumentar el número de hiperparámetros para SVR y el número de valores para estos hiperparámetros. Documenta brevemente el tiempo que emplean y el resultados obtenido.

In [ ]:
params = {"kernel": ["rbf"],
             "C"     : [1,2,3,4,5,6,7,8],
             "gamma" : np.logspace(-9, 9, num=25, base=10)}

#Solución